In [0]:
import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.contrib.eager as tfe

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
from PIL import Image
import numpy as np
import skimage.io as io

In [0]:
tf.enable_eager_execution()

In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.9 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 10 #@param {type:"integer"}

In [0]:
############Define parameters###########################
IMAGE_HEIGHT = 40
IMAGE_WIDTH = 40
IMAGE_DEPTH = 3
NUM_CLASSES = 10

In [0]:
def init_pytorch(shape, dtype=tf.float32, partition_info=None):
  fan = np.prod(shape[:-1])
  bound = 1 / math.sqrt(fan)
  return tf.random.uniform(shape, minval=-bound, maxval=bound, dtype=dtype)


In [0]:
class ConvBN(tf.keras.Model):
  def __init__(self, c_out):
    super().__init__()
    self.conv = tf.keras.layers.Conv2D(filters=c_out, kernel_size=3, padding="SAME", kernel_initializer=init_pytorch, use_bias=False)
    self.bn = tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

  def call(self, inputs):
    return tf.nn.relu(self.bn(self.conv(inputs)))


In [0]:
class ResBlk(tf.keras.Model):
  def __init__(self, c_out, pool, res = False):
    super().__init__()
    self.conv_bn = ConvBN(c_out)
    self.pool = pool
    self.res = res
    if self.res:
      self.res1 = ConvBN(c_out)
      self.res2 = ConvBN(c_out)

  def call(self, inputs):
    h = self.pool(self.conv_bn(inputs))
    if self.res:
      h = h + self.res2(self.res1(h))
    return h

In [0]:
class DavidNet(tf.keras.Model):
  def __init__(self, c=64, weight=0.125):
    super().__init__()
    pool = tf.keras.layers.MaxPooling2D()
    self.init_conv_bn = ConvBN(c)
    self.blk1 = ResBlk(c*2, pool, res = True)
    self.blk2 = ResBlk(c*4, pool)
    self.blk3 = ResBlk(c*8, pool, res = True)
    self.pool = tf.keras.layers.GlobalMaxPool2D()
    self.linear = tf.keras.layers.Dense(10, kernel_initializer=init_pytorch, use_bias=False)
    self.weight = weight

  def call(self, x, y):
    h = self.pool(self.blk3(self.blk2(self.blk1(self.init_conv_bn(x)))))
    h = self.linear(h) * self.weight
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=h, labels=y)
    loss = tf.reduce_sum(ce)
    correct = tf.reduce_sum(tf.cast(tf.math.equal(tf.argmax(h, axis = 1), y), tf.float32))
    return loss, correct


In [0]:
# Loading the Dataset and then Normalizing it.

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
len_train, len_test = len(x_train), len(x_test)
y_train = y_train.astype('int64').reshape(len_train)
y_test = y_test.astype('int64').reshape(len_test)

train_mean = np.mean(x_train, axis=(0,1,2))
train_std = np.std(x_train, axis=(0,1,2))

normalize = lambda x: ((x - train_mean) / train_std).astype('float32') # todo: check here
pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')

x_train = normalize(pad4(x_train))
x_test = normalize(x_test)

170500096/170498071 [==============================] - 13s 0us/step


In [0]:
# Defining the Optimization Strategies.

model = DavidNet()
batches_per_epoch = len_train//BATCH_SIZE + 1

lr_schedule = lambda t: np.interp([t], [0, (EPOCHS+1)//5, EPOCHS], [0, LEARNING_RATE, 0])[0]
global_step = tf.train.get_or_create_global_step()
lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE
opt = tf.train.MomentumOptimizer(lr_func, momentum=MOMENTUM, use_nesterov=True)
data_aug = lambda x, y: (tf.image.random_flip_left_right(tf.random_crop(x, [32, 32, 3])), y)


In [0]:
"""## Mount the google drive"""

from google.colab import drive
drive.mount('/content/gdrive')

"""## Specify the path to save the tf record"""

train_tfrecords_filename = 'gdrive/My Drive/EVA/tf_records_datasets_folder/Training_tf_records.tfrecords'
test_tfrecords_filename = 'gdrive/My Drive/EVA/tf_records_datasets_folder/Testing_tf_records.tfrecords'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
"""## TFRecord converter functions"""

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def convert_data_to_tf(data, labels, tfrecords_filename):
  with tf.python_io.TFRecordWriter(tfrecords_filename) as record_writer:
    num_entries_in_batch = len(labels)
    for i in range(num_entries_in_batch):
      example = tf.train.Example(features=tf.train.Features(
        feature={
          'image': _bytes_feature(data[i].tobytes()),
          'label': _int64_feature(labels[i])
        }))
      record_writer.write(example.SerializeToString())    

In [0]:
convert_data_to_tf(x_train, y_train,train_tfrecords_filename)
convert_data_to_tf(x_test,y_test,test_tfrecords_filename)

Generating gdrive/My Drive/EVA/tf_records_datasets_folder/Training_tf_records.tfrecords
Generating gdrive/My Drive/EVA/tf_records_datasets_folder/Testing_tf_records.tfrecords


In [0]:

"""## TFRecod reader functions"""

def get_each_record(serialized_example, isTraining = True):
  features = tf.parse_single_example(
    serialized_example,
    features={
      'image': tf.FixedLenFeature([], tf.string),
      'label': tf.FixedLenFeature([], tf.int64),
    })
  
  image = features['image']
  image = tf.decode_raw(image, tf.float32)

  if(isTraining):
    image = tf.reshape(image, [IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH])
  else:
    image = tf.reshape(image, [32, 32, IMAGE_DEPTH])
  
  label = tf.cast(features['label'], tf.int64)
  return image, label

def fetch_tf_record_data(file_name, isTraining = True):
  dataset = tf.data.TFRecordDataset(filenames=file_name)
  dataset = dataset.map(lambda x: get_each_record(x, isTraining))
  return dataset    


In [0]:
"""## Read test TFRecord data"""
test_set = fetch_tf_record_data(test_tfrecords_filename, isTraining=False).batch(BATCH_SIZE)


In [0]:
t = time.time()

for epoch in range(EPOCHS):
  train_loss = test_loss = train_acc = test_acc = 0.0
  
  #### Read training TFRecords data
  train_set = generate_input_fn(train_tfrecords_filename).map(data_aug).shuffle(len_train).batch(BATCH_SIZE).prefetch(1)
  tf.keras.backend.set_learning_phase(1)

  for (x, y) in tqdm(train_set):
    with tf.GradientTape() as tape:
      loss, correct = model(x, y)

    var = model.trainable_variables
    grads = tape.gradient(loss, var)
    for g, v in zip(grads, var):
      g += v * WEIGHT_DECAY * BATCH_SIZE
    opt.apply_gradients(zip(grads, var), global_step=global_step)

    train_loss += loss.numpy()
    train_acc += correct.numpy()

  tf.keras.backend.set_learning_phase(0)
  for (x, y) in test_set:
    loss, correct = model(x, y)
    test_loss += loss.numpy()
    test_acc += correct.numpy()
    
  print('epoch:', epoch+1, 'lr:', lr_schedule(epoch+1), 'train loss:', train_loss / len_train, 'train acc:', train_acc / len_train, 'val loss:', test_loss / len_test, 'val acc:', test_acc / len_test, 'time:', time.time() - t)



epoch: 1 lr: 0.2 train loss: 1.4817430712890625 train acc: 0.45824 val loss: 1.4456253723144532 val acc: 0.538 time: 57.82213616371155



epoch: 2 lr: 0.4 train loss: 0.8607685327148438 train acc: 0.69964 val loss: 0.7881681182861328 val acc: 0.7355 time: 93.90614032745361



epoch: 3 lr: 0.35000000000000003 train loss: 0.5988113024902344 train acc: 0.79144 val loss: 0.595013542175293 val acc: 0.8006 time: 130.015855550766



epoch: 4 lr: 0.30000000000000004 train loss: 0.4414110906982422 train acc: 0.84882 val loss: 0.711836083984375 val acc: 0.7751 time: 166.51843857765198



epoch: 5 lr: 0.25 train loss: 0.35802641662597656 train acc: 0.87668 val loss: 0.46467929229736327 val acc: 0.8439 time: 202.85448956489563



epoch: 6 lr: 0.2 train loss: 0.2994898861694336 train acc: 0.89708 val loss: 0.36325263290405274 val acc: 0.8775 time: 239.28970384597778



epoch: 7 lr: 0.15000000000000002 train loss: 0.2515758610534668 train acc: 0.91398 val loss: 0.34982210540771486 val acc: 0.8815 time: 275.763587474823



epoch: 8 lr: 0.09999999999999998 train loss: 0.21012117057800292 train acc: 0.92912 val loss: 0.3048540916442871 val acc: 0.8944 time: 312.16518545150757



epoch: 9 lr: 0.04999999999999999 train loss: 0.17341193649291992 train acc: 0.94106 val loss: 0.285727889251709 val acc: 0.9053 time: 348.56050848960876



epoch: 10 lr: 0.0 train loss: 0.14761189483642578 train acc: 0.951 val loss: 0.2545848598480225 val acc: 0.9136 time: 384.9460530281067
